In [1]:
import pandas as pd

In [2]:
data_url = './make_file/열선_위경도값(중앙값_고도_포함).csv'

In [3]:
data = pd.read_csv(data_url)
data.head(1)

,Unnamed: 0,행정동,노선명,기점,종점,열선연장(m) (1차로 기준),차로수,기점_좌표_위도,기점_좌표_경도,종점_좌표_위도,종점_좌표_경도,열선_중앙_위도,열선_중앙_경도,시작점_고도,종료점_고도,고도의_차이
0,0,구의2동,동의초 통학로,구의2동 34-8,구의2동 81-3,278,2,37.552512,127.095553,37.553039,127.097008,37.552775,127.09628,49.2244,68.678513,19.454113


In [4]:
data.columns

Index(['Unnamed: 0', '행정동', '노선명', '기점', '종점', '열선연장(m) (1차로 기준)', '차로수',
       '기점_좌표_위도', '기점_좌표_경도', '종점_좌표_위도', '종점_좌표_경도', '열선_중앙_위도', '열선_중앙_경도',
       '시작점_고도', '종료점_고도', '고도의_차이'],
      dtype='object')

In [5]:
# 예제 데이터 (배열)
start_lat = data['기점_좌표_위도'].to_list()
start_lon = data['기점_좌표_경도'].to_list()
end_lat = data['종점_좌표_위도'].to_list()
end_lon = data['종점_좌표_경도'].to_list()
different_alt = data['고도의_차이'].to_list()

In [6]:
print(f'start_lat 길이 : {len(start_lat)}')
print(f'start_lon 길이 : {len(start_lon)}')
print(f'end_lat 길이 : {len(end_lat)}')
print(f'end_lon 길이 : {len(end_lon)}')
print(f'different_alt 길이 : {len(different_alt)}')

start_lat 길이 : 41
start_lon 길이 : 41
end_lat 길이 : 41
end_lon 길이 : 41
different_alt 길이 : 41


In [7]:
import math

# 지구 반지름 (미터)
R = 6371000   

In [8]:
# 결과 저장 리스트
road_slope = []

In [9]:
for i in range(len(start_lat)):
    # 위경도를 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(math.radians, [start_lat[i], start_lon[i], end_lat[i], end_lon[i]])

    # 위도, 경도의 차이
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine 공식으로 수평 거리(d) 계산
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c  # 두 지점 간 수평 거리 (m)

    # 고도 차이 (h)
    h = different_alt[i]  

    # 경사각(θ) 계산 (라디안 → 도 변환)
    theta = math.degrees(math.atan2(h, d))

    # 결과 저장
    road_slope.append(theta)

    if(i < 5) :
        # 디버깅 출력
        print(f"Point {i + 1}: d = {d:.4f} m, h = {h:.4f} m, θ = {theta:.4f}°")


Point 1: d = 141.0682 m, h = 19.4541 m, θ = 7.8519°
Point 2: d = 156.2466 m, h = 16.0445 m, θ = 5.8630°
Point 3: d = 0.0000 m, h = 0.0000 m, θ = 0.0000°
Point 4: d = 69.3193 m, h = 6.8931 m, θ = 5.6788°
Point 5: d = 0.0000 m, h = 0.0000 m, θ = 0.0000°


In [10]:
print(f'변환한 road_slope의 길이 : {len(road_slope)}')

변환한 road_slope의 길이 : 41


In [11]:
new_add_df = pd.DataFrame({
    '경사각' : road_slope
})

In [12]:
data['경사각'] = new_add_df

In [13]:
data.columns

Index(['Unnamed: 0', '행정동', '노선명', '기점', '종점', '열선연장(m) (1차로 기준)', '차로수',
       '기점_좌표_위도', '기점_좌표_경도', '종점_좌표_위도', '종점_좌표_경도', '열선_중앙_위도', '열선_중앙_경도',
       '시작점_고도', '종료점_고도', '고도의_차이', '경사각'],
      dtype='object')

In [14]:
new_order = [
    '행정동', '노선명', '기점', '종점', '기점_좌표_위도', '기점_좌표_경도', '종점_좌표_위도', '종점_좌표_경도',
    '열선_중앙_위도', '열선_중앙_경도', '시작점_고도', '종료점_고도', '고도의_차이', '경사각'
] + [col for col in data.columns if col not in [
    '행정동', '노선명', '기점', '종점', '기점_좌표_위도', '기점_좌표_경도', '종점_좌표_위도', '종점_좌표_경도',
    '열선_중앙_위도', '열선_중앙_경도', '시작점_고도', '종료점_고도', '고도의_차이', '경사각'
]]

# 컬럼 순서 재배열
data = data[new_order]


In [15]:
data.columns

Index(['행정동', '노선명', '기점', '종점', '기점_좌표_위도', '기점_좌표_경도', '종점_좌표_위도',
       '종점_좌표_경도', '열선_중앙_위도', '열선_중앙_경도', '시작점_고도', '종료점_고도', '고도의_차이', '경사각',
       'Unnamed: 0', '열선연장(m) (1차로 기준)', '차로수'],
      dtype='object')

In [16]:
data['경사각']

0      7.851888
1      5.862993
2      0.000000
3      5.678818
4      0.000000
5      0.000000
6     10.295554
7      9.561299
8      9.064065
9     10.685221
10     8.257422
11     7.648155
12     8.226913
13     8.263302
14    14.737624
15     7.486803
16    15.783514
17     0.832585
18    11.339577
19     2.436242
20     5.858782
21     0.130472
22     0.130472
23     0.520453
24     1.209439
25     1.016910
26     2.107477
27     0.891187
28    20.624165
29    10.826622
30     0.500065
31     1.175963
32     9.467428
33     5.590429
34     1.275380
35     2.019389
36     0.013194
37    10.920562
38     4.715757
39     0.543819
40     1.653873
Name: 경사각, dtype: float64

In [19]:
save_data_url = './make_file/열선_위경도값(중앙값_고도_경사각_포함).csv'
data.to_csv(save_data_url, encoding='euc-kr')

print('파일 저장 완료')

파일 저장 완료
